<a href="https://colab.research.google.com/github/itrail/MovieRecommendationSystem/blob/main/Expert_System_Movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install experta

  Created wheel for frozendict: filename=frozendict-1.2-cp37-none-any.whl size=3150 sha256=958bcdfc70e2cb9b33605103a378412acd92fd661d88cf6e117562cf28be314d
  Stored in directory: /root/.cache/pip/wheels/6c/6c/e9/534386165bd12cf1885582c75eb6d0ffcb321b65c23fe0f834
Successfully built frozendict


In [ ]:
from random import choices
from experta import *

In [ ]:
import json

with open('movies.json', 'r') as f:
  json_object = json.load(f)
  f.close()

In [ ]:
#importing all genres from json file
genres = []
i = 0
for item in json_object:
  i= i + 1
  try:
    genres.append(item['genres'][0].lower())
    genres = list(dict.fromkeys(genres))
    print(item['genres'][0])
  except:
    continue

In [ ]:
genres

['action',
 'thriller',
 'animation',
 'documentary',
 'adventure',
 'fantasy',
 'mystery',
 'horror',
 'comedy',
 'music',
 'family',
 'crime',
 'drama',
 'romance',
 'science fiction',
 'war',
 'tv movie',
 'western',
 'history']

In [ ]:
import random

class Movies_Ratings(Fact):
    """Info about the ratings of movies."""
    pass
    
    #ustalenie wag poczatkowych
    def __init__(self):
      self.weights_of_genres = [50 for i in range(len(genres))]#domyslnie wagi wynoszą 50%
      self.counters = [0 for i in range(len(genres))]
      self.watched_movies = []

    #drukowanie wag
    def print_weights(self):
      print(self.weights_of_genres)

    def print_counters(self):
      print(self.counters)

    def rate_movie(self, film_to_rate):
      try:
        genre_to_rate = film_to_rate[0]['genres'][0].lower()
        print("Oceń film: " + film_to_rate[0]['title'])
        print("Gatunek: " + film_to_rate[0]['genres'][0])

        index_to_rate = genres.index(genre_to_rate)
        flag=True
        while flag == True:
          rating = int(input('Podaj ocenę: [0-10] '))#on the webpage rating will be implemented as 'stars choosing'
          if rating in range(0, 11): #0-10
            flag = False
        if self.counters[index_to_rate] == 0:
          self.weights_of_genres[index_to_rate] = rating*10
        else:
          self.weights_of_genres[index_to_rate] = ((self.counters[index_to_rate]-1) * self.weights_of_genres[index_to_rate] + (rating*10))/self.counters[index_to_rate]
        self.counters[index_to_rate] += 1
      except:
        print("Movie without genre")

In [ ]:
user_Ratings = Movies_Ratings()

In [ ]:
class Movie_Expert(KnowledgeEngine):
  @Rule(Movies_Ratings())
  def draw_movies(self):
    #print("Wybrane gatunki filmów: ")
    r = random.choices(genres, Movie.weights_of_genres, k=3)
    #for elem in r:
      #print(elem)
    
    print("\tWybrane filmy: ")
    for elem in r:
      temp = []
      for item in json_object:
        try:
          if item['genres'][0].lower() == elem:
            temp.append(item)
        except:
          continue
      flag = True
      while(flag==True):
        irand = random.randrange(0, len(temp))
        if temp[irand] not in Movie.watched_movies:
          flag = False
          print(temp[irand]['title'])

In [ ]:
#drawing 3 movies recommended for user
engine = Movie_Expert()
engine.reset()
engine.declare(user_Ratings)
engine.run()

	Wybrane filmy: 
The Bone Collector
5 Days of War
The League of Extraordinary Gentlemen


In [ ]:
#rating - weights changing
#zmieniamy wagi ile chcemy

film_to_rate = random.choices(json_object)#random movie. On the website, the user selects the movie he saw and its genre is determined.
user_Ratings.watched_movies.append(film_to_rate)
user_Ratings.rate_movie(film_to_rate)

Oceń film: Fist of Fury
Gatunek: Action
Podaj ocenę: [0-10] 3


In [ ]:
user_Ratings.print_weights()

[30, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50, 50]


In [ ]:
user_Ratings.print_counters()

[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [ ]:
user_Ratings.watched_movies

[[{'genres': ['Action'],
   'id': '11713',
   'overview': "Chen Zhen returns to the international compound of China only to learn of his beloved teacher's death. This is compounded by the continual racist harassment by the Japanese population in the area. Unlike his friends, he confronts it head on with his mastery of martial arts while investigating his teacher's murder.",
   'poster': 'https://image.tmdb.org/t/p/w500/dlcipCOa9hlfBBz7kCAyjsf3q0E.jpg',
   'release_date': 70070400,
   'title': 'Fist of Fury'}]]

In [ ]:
#destroing an object
del user_Ratings